In [62]:
import mne
import json
import torch
mne.set_log_level('ERROR')
mne.cuda.init_cuda(verbose=True)
import pandas as pd
import numpy as np

CUDA not enabled in config, skipping initialization


In [63]:
#create a lisst of every csv and its corresponding edf
csv_file_paths=[r"C:\Users\uddha\OneDrive\Desktop\SS Learning\Se\SeizureSense\TempleDataset\aaaaaaac_s001_t000 (1).csv"]
edf_file_paths=[r"C:\Users\uddha\Downloads\chb02_19.edf"]

In [32]:
#Okay so i needd to create a dicitionary for each channel becayse thats the way its in a row with it being a 
# different row for each channel that has a new label in it  
# ex: 
# FP1-F7,0.0000,36.8868,bckg,1.0000
#FP1-F7,36.8868,183.3055,cpsz,1.0000
#FP1-F7,183.3055,301.0000,bckg,1.0000
#all of these are one channel but its displayed as rows. 
# i also need to check if in the channel name it has a LE or an AR reference and if the channel doesnt have 
#19 non referenced channels I drop it from the list of csv paths and edf paths
#so i have to create a dictionary for each channel and make sure it has the same label for each of the time step
#and make sure they are all the same plus or minus 5 seceonds then create the real dictionary with the format: 
#'CHB-MIT/CHB_Database/chb01/chb01_03.edf': [(0, 2396, 'interictal'), (2396, 2996, 'preictal'), (2996, 3036, 'ictal'), (3036, 3600, 'interictal')],
#and append that to the master dictionary that gets created

Creating a Dictionary for temple dataset


In [64]:
def validate_edf_channels(raw, required_channels=19):
    
    channels_to_drop = [ch_name for ch_name in raw.ch_names if "LE" in ch_name or "AR" in ch_name]
    num_valid_channels = len(raw.ch_names) - len(channels_to_drop)
    if len(channels_to_drop)>0:
        raw.drop_channels(channels_to_drop)
    if num_valid_channels >= required_channels:
        return True
    else:
        return False

def calculate_seizure_times_by_channel(df):
    
    seizure_df = df[df['label'] == 'cpsz']
    seizure_start_times = []
    seizure_end_times = []
    
    # Group by channel and process each group
    for channel, group in seizure_df.groupby('channel'):
        # Sort the group by start time to ensure chronological order
        group = group.sort_values('start_time')
        
        # Initialize temporary lists to store individual seizure times for the current channel
        temp_start_times = []
        temp_end_times = []
        
        for _, row in group.iterrows():
            # Append start and end times to temporary lists
            temp_start_times.append(row['start_time'])
            temp_end_times.append(row['stop_time'])
        
        # Calculate the average start and end times for seizures in this channel
        if temp_start_times:
            seizure_start_times.append(np.mean(temp_start_times))
            seizure_end_times.append(np.mean(temp_end_times))
    
    # Round the average times to nearest second and ensure they are distinct
    seizure_start_times = [round(time) for time in set(seizure_start_times)]
    seizure_end_times = [round(time) for time in set(seizure_end_times)]
    
    return seizure_start_times, seizure_end_times

def create_labels_dict(edf_file_path, seizure_start_times, seizure_end_times, total_duration):
    # Initialize the list to store the time periods and labels
    labels = []
    
    # Start with an 'interictal' period if the first seizure starts after 0
    if seizure_start_times and seizure_start_times[0] > 600:
        labels.append((0, seizure_start_times[0] - 600, 'interictal'))
    
    # Process each seizure
    for start, end in zip(seizure_start_times, seizure_end_times):
        preictal_start = max(start - 600, 0)  # Ensure preictal doesn't start before 0
        if labels and labels[-1][1] < preictal_start:
            # Add an 'interictal' period before 'preictal' if there's a gap
            labels.append((labels[-1][1], preictal_start, 'interictal'))
        labels.append((preictal_start, start, 'preictal'))
        labels.append((start, end, 'ictal'))
    
    # End with an 'interictal' period from the last 'ictal' or 'preictal' to the end
    if labels:
        final_end_time = labels[-1][1]
        if final_end_time < total_duration:
            labels.append((final_end_time, total_duration, 'interictal'))
    else:
        # If there are no seizures, mark the entire duration as 'interictal'
        labels.append((0, total_duration, 'interictal'))
    
    return {edf_file_path: labels}



def create_dictionaries(edf_file_path,csv_file_path):

        
    seizure_start_times, seizure_end_times=calculate_seizure_times_by_channel(df)
    labels_dict=create_labels_dict(edf_file_path, seizure_start_times, seizure_end_times, total_duration)
    return labels_dict
    
    
    
    

In [65]:
master_labels_dict = {}
for edf_file_path, csv_file_path in zip(edf_file_paths,csv_file_paths):
    raw=mne.io.read_raw_edf(edf_file_path,preload=True,verbose=False)
    print(raw.ch_names)
    raw=raw.resample(200,npad='auto')
    total_duration=raw.n_times/200
    
    df=pd.read_csv(csv_file_path,comment="#")
    
    if not validate_edf_channels(raw):
        csv_file_paths.remove(csv_file_path)
        edf_file_paths.remove(edf_file_path)
        continue
    labels_dict=create_dictionaries(edf_file_path, csv_file_path)
    master_labels_dict.update(labels_dict)
    
print(master_labels_dict)

C:\Users\uddha\AppData\Local\Temp\ipykernel_15532\2608346524.py:3: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw=mne.io.read_raw_edf(edf_file_path,preload=True,verbose=False)


['FP1-F7', 'F7-T7', 'T7-P7', 'P7-O1', 'FP1-F3', 'F3-C3', 'C3-P3', 'P3-O1', 'FP2-F4', 'F4-C4', 'C4-P4', 'P4-O2', 'FP2-F8', 'F8-T8', 'T8-P8-0', 'P8-O2', 'FZ-CZ', 'CZ-PZ', 'P7-T7', 'T7-FT9', 'FT9-FT10', 'FT10-T8', 'T8-P8-1']
{'C:\\Users\\uddha\\Downloads\\chb02_19.edf': [(0, 37, 'preictal'), (37, 160, 'ictal'), (160, 3600.0, 'interictal')]}


In [57]:
raw=mne.io.read_raw_edf(r"C:\Users\uddha\Downloads\chb02_19.edf",preload=True)
file_path=r"C:\Users\uddha\Downloads\aaaaaaac_s001_t001.edf"
print(raw.ch_names)
print(validate_edf_channels(raw))

['FP1-F7', 'F7-T7', 'T7-P7', 'P7-O1', 'FP1-F3', 'F3-C3', 'C3-P3', 'P3-O1', 'FP2-F4', 'F4-C4', 'C4-P4', 'P4-O2', 'FP2-F8', 'F8-T8', 'T8-P8-0', 'P8-O2', 'FZ-CZ', 'CZ-PZ', 'P7-T7', 'T7-FT9', 'FT9-FT10', 'FT10-T8', 'T8-P8-1']
['FP1-F7', 'F7-T7', 'T7-P7', 'P7-O1', 'FP1-F3', 'F3-C3', 'C3-P3', 'P3-O1', 'FP2-F4', 'F4-C4', 'C4-P4', 'P4-O2', 'FP2-F8', 'F8-T8', 'T8-P8-0', 'P8-O2', 'FZ-CZ', 'CZ-PZ', 'P7-T7', 'T7-FT9', 'FT9-FT10', 'FT10-T8', 'T8-P8-1']
True


Segmenting the data

In [72]:
def preprocessing(raw):
    raw=raw.resample(200)
    eeg_picks = mne.pick_types(raw.info, meg=False, eeg=True)


    freqs = (60)
    raw_notch = raw.copy().notch_filter(freqs=freqs, picks=eeg_picks)
    
    raw.notch_filter(freqs=freqs, picks=eeg_picks)  # Apply notch filter
    raw.filter(l_freq=30, h_freq=None, fir_design='firwin', filter_length='auto', phase='zero', fir_window='hamming')
    numpy_array=raw.get_data()
    segment_tensor=torch.from_numpy(numpy_array).float().unsqueeze(0).unsqueeze(0)
    return(segment_tensor)

In [75]:
file_path=r"C:\Users\uddha\Downloads\aaaaaaac_s001_t001.edf"

torch.Size([1, 1, 33, 200])


In [ ]:
for edf_file_path in edf_file_paths:
    raw=mne.io.read_raw_edf(edf_file_path,preload=True)
    tensor=preprocessing(raw)
    #save one seceond segments as pt files
        
        
        